## Notebook to take HCP-YA Behavioral data dictionary and corresponding data and request a new structure and reshape data for NDA

In [ ]:
import pandas as pd
import re, datetime
import numpy as np
import os

In [ ]:
#this is the data dictionary downloaded from IntraDB
dictionya=pd.read_csv('CanonicalDataDictionaryCSV.csv', encoding = 'ISO-8859-1')
print(dictionya.shape)
dictionya=dictionya.drop_duplicates(subset='columnHeader')
print(dictionya.shape)


In [ ]:
#initialize NDA data dictionary variables for structure request and crosswalk. Note that we have to keep YAElement for Element because there will
#distinct from NDA element, due to variable name length restrictions in the latter and ndar_subjects requirements
dictionya['YAElement']=dictionya.columnHeader
dictionya['Required']='Recommended'
dictionya['Data Type']=dictionya.dictType
dictionya['nda_description']=dictionya.description
dictionya['Notes']=dictionya['values']
dictionya['Size']=''
dictionya['Value Range']=dictionya['values']
dictionya.loc[dictionya.description.isnull()==True,'nda_description']=dictionya.fullDisplayName
dictionya['hcp_description']=dictionya.nda_description
dictionya.loc[dictionya.dictType=='$','Data Type']='Float'
dictionya.loc[dictionya['Data Type']=='Boolean','Data Type']='String'

#fix one incorrect datatype
dictionya.loc[dictionya['YAElement']=='Correction','Data Type']='Float'



In [ ]:
#load data fieldnames from data so we can 
#subset to fields that are available for download on IntraDB:
#keep 5 subjects' data so that we can see the format when spot checking output

d1=pd.read_csv('data/RESTRICTED_plenzini_3_22_2022_11_34_54.csv',nrows=5)
d2=pd.read_csv('data/unrestricted_plenzini_3_22_2022_11_34_38.csv',nrows=5)
d3=pd.read_csv('data/unrestricted_plenzini_3_22_2022_11_35_0.csv',nrows=5)
d=pd.concat([d1.transpose(),d2.transpose(),d3.transpose()],axis=0)
d=d.reset_index()
print(d.shape)
d=d.drop_duplicates() #three Age bucketing variables from different sources
print(d.shape)
d=d.rename(columns={'index':'YAElement'})
d.head()

In [ ]:
#merge together for intersection of datadictionary elements and data elements
#there shouldn't be any duplicates because we already got rid of the duplicate 'Age' bucket variables (not needed because NDA uses age in months anyway)

a=pd.merge(dictionya,d,on='YAElement',how='right')
print(a.shape)
a.head()
#a.columns
a.loc[a.YAElement=='Age_in_Yrs']
a=a.drop_duplicates(subset='YAElement')
print(a.shape)


In [ ]:
#patch notes and values, since this is faster than parsing all of the exceptions to trends
patch=pd.read_csv('ValuePatch.csv', encoding = 'ISO-8859-1')
updated = a.merge(patch, how='left', on=['YAElement'], suffixes=('', '_new'))
updated['Value Range'] = np.where(pd.notnull(updated['Value Range_new']), updated['Value Range_new'], updated['Value Range'])
updated['Notes'] = np.where(pd.notnull(updated['Notes_new']), updated['Notes_new'], updated['Notes'])
updated=updated.loc[~(updated.YAElement=='Age')]
updated.loc[updated.Notes=='_','Notes']==''
#updated[['Value Range','Notes']]



In [ ]:
#these will be converted to NDA variables during data manipulation.  Right now we're just preparing crosswalk
#so don't need them (they'll get added in mandatory variables part next).
updated=updated.loc[~(updated.YAElement.isin(['Aquisition','Age_in_Yrs','Gender','Subject','subjectkey']))]#=updated.loc[~(updated.Element=='Age')]


In [ ]:
#Trim YA element names for NDA (need to be 30 characters)
#replacements
updated['Element']=updated.YAElement

subs={'Weekday':'Wkdy',
    'Weekend':'Wknd',
    'Cooler':'Cool',
    'Language':'Lang',
    'anteriorcingulate':'antcingul',
    'Gambling':'Gamb',
    'Median':'Med',
   'Smaller':'Small',
   'Larger':'Lrg',
   'Difficulty':'Diffic',
   'Level':'Lvl',
   'Tobacco':'Tob',
   'Random':'Rand',
   'Nontarget':'NTarg',
   'Target':'Targ'                                        
   }

for word, abbrev in subs.items():
    updated['Element'] = updated.Element.str.replace(word,abbrev)

updated['shortelemstrlength']=updated.Element.str.len()
print("any element strings >30?")
print(updated.loc[updated.shortelemstrlength>30])

#check that you didnt create non-unique variable name by comparing Ns
print(updated.shape)
print(len(updated.Element.unique()))

In [ ]:
#add the NDA fields (adding them to crosswalk...the rename/reformat in the data itself will happen later)
structuremandatory1=pd.DataFrame({'YAElement':['nda_guid','Subject','Aquisition','Age_in_Yrs','Gender','dummy8'],
                                 'Element': ['subjectkey','src_subject_id','interview_date','interview_age','sex','race'], 
                                 'Required': ['Required','Required','Required','Required','Required','Required'],
                                 'Data Type': ['GUID','String','Date','Integer','String','String'],
                                 'Size': ['','20','','','20','30'],
                                 'hcp_description':['Pseudo GUID',"HCP Subject ID",'Aquisition Quarter converted to Date. MM/DD/YYYY','Age in Years converted to Age in months','Sex of subject at birth','Mandatory NDA categories for race of study subject.  See HCPYA_Race in hcpya01 structure instead'],
                                 'nda_description':['The NDAR Global Unique Identifier (GUID) for research subject',"Subject ID how it's defined in lab/project",'Date on which the interview/genetic test/sampling/imaging/biospecimen was completed. MM/DD/YYYY','Age in months at the time of the interview/test/sampling/imaging.','Sex of subject at birth','Race of study subject'],
                                 'Value Range':['NDAR*','','','0 :: 1260','M;F; O; NR',''],  
                                 'Notes':['','','','','','']})

structuremandatory2=pd.DataFrame({'YAElement':['dummy1','dummy2','dummy3','dummy4','dummy5','dummy6','dummy7'],
                                  'Element':['phenotype','phenotype_description','twins_study','sibling_study','family_study','comments_misc','sample_taken'],
                                  'Required': ['Required','Required','Required','Required','Required','Required','Required'],
                                  'Data Type':['String','String','String','String','String','String','String'],
                                  'hcp_description':['hardcoded dummy variable','hardcoded dummy variable','hardcoded dummy variable','hardcoded dummy variable','hardcoded dummy variable','hardcoded dummy variable','hardcoded dummy variable'],
                                  'nda_description':['Phenotype/diagnosis for the subject','Description of the phenotype for the subject','Is this study of twins?','Was it sibling study? Study of sibling(s) of autistic child.','Was it family study? Study of biological mother, biological father and/or sibling of proband','Miscellaneous comments on study, interview, methodology relevant to this form data','Was a sample taken at this interview/during this project time'],
                                  'Notes':['','','','','','',''],
                                  'Value Range':['','','','','','','']
                                                })
#'dummy1':'phenotype',
#'dummy2':'phenotype_description',
#'dummy3':'twins_study',
#'dummy4':'sibling_study',
#'dummy5':'family_study'}
#'dummy6':'comments_misc'
#'dummy7':'sample_taken'

structuremandatory=pd.concat([structuremandatory1,structuremandatory2],axis=0)
#structuremandatory

In [ ]:
#this is not really the final crosswalk...misnomer...sorry
final=pd.concat([structuremandatory,updated[['Required','hcp_description','nda_description','Element','YAElement','Data Type','Size','Notes','Value Range','Value Range_new','Notes_new',0,1,2,3,4]]],axis=0)#
final.head(20)
#inttypes=list(final.loc[final['Data Type']=='Integer'])
a=final.loc[final['Data Type']=='Integer'][['YAElement']]
inttypes=list(a['YAElement'])
#inttypes

In [ ]:
#need to map YAElement to Element in crosswalk
#note that Race (HCP) is not the same as race (nda)
#replace strings
renames={
'ZygosityGT':'zygosity',
'Family_ID':'family_user_def_id',
'Mother_ID':'src_mother_id',
'Father_ID':'src_father_id',
'Ethnicity':'ethnic_group'}

for word, abbrev in renames.items():
    final.loc[final.YAElement==word,'Element']=abbrev

#'ZygositySR' doesn't have a place ndar subjects so it will have to show up in the main table

In [ ]:
final=final.rename(columns={'Element':'Element Name'})

#create the rename list for prepping the data itself later
renames4NDA=dict(zip(final.YAElement,final['Element Name']))


In [ ]:
#now prep data to match dictionary
#load data  fields that are available for download on IntraDB and will go:
d1=pd.read_csv('data/RESTRICTED_plenzini_3_22_2022_11_34_54.csv')
d2=pd.read_csv('data/unrestricted_plenzini_3_22_2022_11_34_38.csv')
d3=pd.read_csv('data/unrestricted_plenzini_3_22_2022_11_35_0.csv')
d2=d2.drop(columns=['Age'])
for i in list(d3.columns)+list(d2.columns)+list(d1.columns):
    if 'Age' in i:
        print(i)

In [ ]:
dataraw=pd.merge(d1,d2,on='Subject',how='inner')
dataraw=pd.merge(dataraw,d3,on='Subject',how='inner')
dataraw.shape
dataraw=dataraw.rename(columns={'Subject':'src_subject_id','Gender':'sex'})
dataraw['interview_age']=dataraw['Age_in_Yrs']*12

dataraw.loc[dataraw.Acquisition=='Q01','interview_date']='08/01/2012'
dataraw.loc[dataraw.Acquisition=='Q02','interview_date']='11/01/2012'
dataraw.loc[dataraw.Acquisition=='Q03','interview_date']='02/01/2013'
dataraw.loc[dataraw.Acquisition=='Q04','interview_date']='05/01/2013'
dataraw.loc[dataraw.Acquisition=='Q05','interview_date']='08/01/2013'
dataraw.loc[dataraw.Acquisition=='Q06','interview_date']='11/01/2013'
dataraw.loc[dataraw.Acquisition=='Q07','interview_date']='02/01/2014'
dataraw.loc[dataraw.Acquisition=='Q08','interview_date']='05/01/2014'
dataraw.loc[dataraw.Acquisition=='Q09','interview_date']='08/01/2014'
dataraw.loc[dataraw.Acquisition=='Q10','interview_date']='11/01/2014'
dataraw.loc[dataraw.Acquisition=='Q11','interview_date']='02/01/2015'
dataraw.loc[dataraw.Acquisition=='Q12','interview_date']='05/01/2015'
dataraw.loc[dataraw.Acquisition=='Q13','interview_date']='08/01/2015'

dataraw['interview_date']=pd.to_datetime(dataraw['interview_date']).dt.strftime('%m/%d/%Y')


In [ ]:
#makes sure inttypes are int types (according to how NDA reads these which is actually a string)
#first double check that type wasn't incorrectly assigned
dataraw[inttypes].to_csv('testints.csv',index=False)

for i in inttypes:
    try:
        dataraw[i]=dataraw[i].round().fillna(-9999).astype(int).astype(str).str.replace('-9999','')
    except:
        print('problem with',i)
    
dataraw=dataraw.drop(columns=['Age_in_Yrs'])



In [ ]:
#rename columns to match NDA elements
dataraw=dataraw.rename(columns=renames4NDA)


In [ ]:
#not renamed yet
dataraw.Race.value_counts()

#not created yet
#dataraw.race.value_counts()

#annotation
final.loc[final['Element Name']=='Race']

#not added yet
#final.loc[final['Element Name']=='race']


In [ ]:
for i in list(final['Element Name']):
    if i not in list(dataraw.columns): 
        print('in annotation only:',i)
        
for i in list(dataraw.columns):
    if i not in list(final['Element Name']): 
        print('in data only:',i)        

In [ ]:
#add psuedoguids (subjectkey) to data
pseudos=pd.read_csv('data/hcpya_guid_list.csv')
pseudos.columns=['src_subject_id','subjectkey']
pseudos.head()

alldata=pd.merge(pseudos,dataraw,on='src_subject_id',how='right',indicator=True)
print(alldata._merge.value_counts())
alldata=alldata.drop(columns=['_merge'])

#dataraw.head()

In [ ]:
alldata.columns

In [ ]:
#condition comments_misc on HasGT and race on Race
alldata['comments_misc']=''
alldata.loc[alldata.HasGT==True,'comments_misc']="Genotypes were derived from saliva and/or whole blood and uploaded to dbGAP under GUID identifiers"

alldata.HasGT.value_counts()
alldata.comments_misc.value_counts()

#races are not coded the same across databases
alldata['race']=alldata.replace({'Race':
                                       {'Am. Indian/Alaskan Nat.':'American Indian/Alaska Native',
                                        'Black or African Am.':'Black or African American',
                                        'White':'White',
                                        'Asian/Nat. Hawaiian/Othr Pacific Is.':'More than one race',
                                        'More than one':'More than one race',
                                        'Unknown or Not Reported':'Unknown or not reported'}})['Race']




In [ ]:

#############################
#race (NDA) and Race are not the same
#rename Race to make it clear that Race comes from HCP
#############################
print(alldata.Race.value_counts())
print(alldata.race.value_counts())

final.loc[final['YAElement']=='Race','Element Name']='HCPYA_Race'
alldata=alldata.rename(columns={'Race':'HCPYA_Race'})



In [ ]:
newstructure=alldata.drop(columns=['race','ethnic_group',
       'zygosity', 'family_user_def_id', 'src_mother_id', 'src_father_id','comments_misc'])
print('#variables in new structure:',len(newstructure.columns))
#write out csv for validation
filePath='HCPYA_prepped/hcpya01.csv'

if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

with open(filePath,'a') as f:
    f.write("hcpya01,1\n")
    newstructure.to_csv(f,index=False)




In [ ]:
#move  race ethnic_group to ndar_subject01
#separate into ndar subjects and all other data
ndar=alldata[['src_subject_id', 'subjectkey','interview_date','interview_age','sex','race','ethnic_group', 
       'zygosity', 'family_user_def_id', 'src_mother_id', 'src_father_id','comments_misc']].copy()


ndar.head()

In [ ]:
ndar.race.value_counts() #different from HCP race

In [ ]:
ndar.loc[:,'phenotype']=pd.Series(("Healthy Subject" for i in range(ndar.shape[0])),index=ndar.index)
ndar.loc[:,'phenotype_description']=pd.Series(("No diagnosed history of neurologic or major psychiatric disorder" for i in range(ndar.shape[0])),index=ndar.index)
ndar.loc[:,'twins_study']=pd.Series(("No" for i in range(ndar.shape[0])),index=ndar.index)
ndar.loc[:,'sibling_study']=pd.Series(("No" for i in range(ndar.shape[0])),index=ndar.index)
ndar.loc[:,'family_study']=pd.Series(("No" for i in range(ndar.shape[0])),index=ndar.index)
ndar.loc[:,'sample_taken']=pd.Series(("No" for i in range(ndar.shape[0])),index=ndar.index)

ndar=ndar.rename(columns={'Family_ID':'family_user_def_id','Father_ID':'src_father_id','Mother_ID':'src_mother_id'})



ndar['zygosity']=ndar.replace({'zygosity':
                                     {'MZ':'monozygous',
                                      'DZ':'dizygous'}})['zygosity']

#ethnicities are not coded the same across databases
ndar['ethnic_group']=ndar.replace({'ethnic_group':
                                           {'Hispanic/Latino':'Hispanic or Latino',
                                            'Not Hispanic/Latino':'Not Hispanic or Latino',
                                            'Unknown or Not Reported':'unknown or not reported'}})['ethnic_group']


list(ndar.columns)
ndar.head()

In [ ]:
#write out csv for validation
filePath='HCPYA_prepped/ndar_subject01.csv'
print("#variables in ndar_subjects:",len(ndar.columns))

if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

with open(filePath,'a') as f:
    f.write("ndar_subject,1\n")
    ndar.to_csv(f,index=False)



In [ ]:
#clean up the crosswalk and assign structures.  
#then finalize the new structure request variable subset
crosswalk=final.drop(columns=[0,1,2,3,4])
#assign crosswalk elements to structures
crosswalk['structure']=''
crosswalk.loc[crosswalk['Element Name'].isin(list(ndar.columns)),'structure']='ndar_subject'
crosswalk.loc[crosswalk['Element Name'].isin(['src_subject_id','subjectkey', 'interview_date', 'interview_age', 'sex']),'structure']='All Structures'
crosswalk.loc[crosswalk['structure']=='','structure']='hcpya01'

crosswalk.loc[crosswalk['Element Name'].isin(['Race','ethnic_group']),'Notes']=''
crosswalk.loc[crosswalk['Notes']=='_','Notes']=''


In [ ]:
crosswalk.structure.value_counts()

In [ ]:
#set string lengths for data variables (not ndar_subjects or All structures variables)
b=crosswalk.loc[(crosswalk['Data Type']=='String') & (~(crosswalk.structure.isin(['ndar_subject','All Structures']))) ][['Element Name']]
print('b size:',b.shape)
strvars=list(b['Element Name'])

#strvars=list(crosswalk.loc[crosswalk['Data Type']=='String'][['Element Name']])
mx_dct = {c: newstructure[c].map(lambda x: len(str(x))).max() for c in strvars}
varlengths=pd.Series(mx_dct).sort_values(ascending =False)
v=pd.DataFrame(varlengths.reset_index())
v.columns=['Element Name','newlength']
v.head()
v['newlength']=v.newlength/10
v['newlength']=10*v.newlength.apply(np.ceil).astype('Int64')
v.head()
crosswalk=crosswalk.merge(v,how='left', on=['Element Name'])
crosswalk.columns
crosswalk.loc[(crosswalk.newlength.isnull()==False) & (crosswalk['Data Type']=='String'),'Size']=crosswalk.newlength
crosswalk=crosswalk.drop(columns=['Value Range_new','Notes_new','newlength'])
#crosswalk.head(20)

In [ ]:
crosswalk.to_csv("HCP_YA_Crosswalk.csv",index=False)
print("number of elements in crosswalk:",crosswalk.shape[0])
crosswalk.columns

In [ ]:
newstructurerequest=crosswalk.loc[~(crosswalk['Element Name'].isin(['race','ethnic_group', 'comments_misc', 
       'zygosity', 'family_user_def_id', 'src_mother_id', 'src_father_id','phenotype','phenotype_description',
                                'twins_study','sibling_study','family_study','sample_taken']))].drop(columns=['structure','YAElement','hcp_description'])
newstructurerequest.shape

newstructurerequest=newstructurerequest.rename(columns={'nda_description':'Description'})


In [ ]:
newstructurerequest.to_csv("hcpya01_structuredef.csv",index=False)

In [ ]:
#double check variable counts
for i in list(newstructurerequest['Element Name']):
    if i not in list(newstructure.columns): 
        print('in annotation only:',i)
        
for i in list(newstructure.columns):
    if i not in list(newstructurerequest['Element Name']): 
        print('in data only:',i)        